In [9]:
import pandas as pd
import glob
import regex as re

In [27]:
files = glob.glob("results/*.csv")
dfs = [pd.read_csv(f, index_col=0) for f in files]
files[0]

'results/eval_results_zeroshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv'

In [28]:
for f, df in zip(files, dfs):

    prompt_type=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    model_id=re.findall(r"results/eval_results_\w+_(.+)\.csv", f)[0]
    df['prompt_type'] = prompt_type
    df['model_id'] = model_id

df = pd.concat(dfs)
df

,erl,response,n_nodes,seed,f1_score,edit_distance,prompt_type,model_id
0,relations=[EnrichedRelation(entity='Organisati...,An organisation member's death place is a stre...,3,0,NaN,NaN,zeroshot,NousResearch-Hermes-3-Llama-3.1-8B-GGUF
1,relations=[EnrichedRelation(entity='written wo...,"A written work, which is a part of a multi-vol...",3,1,NaN,NaN,zeroshot,NousResearch-Hermes-3-Llama-3.1-8B-GGUF
2,"relations=[EnrichedRelation(entity='work', rel...",A work stars an actor and has an auteur who is...,3,2,NaN,NaN,zeroshot,NousResearch-Hermes-3-Llama-3.1-8B-GGUF
3,relations=[EnrichedRelation(entity='Organisati...,An organisation member of an organisation has ...,3,3,NaN,NaN,zeroshot,NousResearch-Hermes-3-Llama-3.1-8B-GGUF
4,relations=[EnrichedRelation(entity='Grand Prix...,The Grand Prix has its first driver being a Sp...,3,4,NaN,NaN,zeroshot,NousResearch-Hermes-3-Llama-3.1-8B-GGUF
...,...,...,...,...,...,...,...,...
892,"relations=[EnrichedRelation(entity='person', r...",A person has an alma mater which is a school. ...,10,295,0.000000,0.600000,oneshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
893,relations=[EnrichedRelation(entity='Organisati...,An Organisation member is born in a town and t...,10,296,0.181818,0.384615,oneshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
894,relations=[EnrichedRelation(entity='ethnic gro...,An ethnic group resides in a populated place. ...,10,297,0.000000,0.341463,oneshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
895,relations=[EnrichedRelation(entity='Organisati...,A band has Organisation members with nationali...,10,298,0.461538,0.384615,oneshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF


In [26]:
df_clean=df.dropna()
df_clean

,Unnamed: 0,erl,response,n_nodes,seed,f1_score,edit_distance,prompt_type,model_id
0,0,relations=[EnrichedRelation(entity='Organisati...,An organisation member's death place is a stre...,3,0,0.000000,0.636364,zeroshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
1,1,relations=[EnrichedRelation(entity='written wo...,"A written work, which is a part of a multi-vol...",3,1,0.800000,0.750000,zeroshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
2,2,"relations=[EnrichedRelation(entity='work', rel...",A work stars an actor and has an auteur who is...,3,2,1.000000,0.727273,zeroshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
3,3,relations=[EnrichedRelation(entity='Organisati...,An organisation member of an organisation has ...,3,3,0.333333,0.500000,zeroshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
4,4,relations=[EnrichedRelation(entity='Grand Prix...,The Grand Prix has its first driver being a Sp...,3,4,0.000000,0.500000,zeroshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
...,...,...,...,...,...,...,...,...,...
892,892,"relations=[EnrichedRelation(entity='person', r...",A person has an alma mater which is a school. ...,10,295,0.000000,0.600000,oneshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
893,893,relations=[EnrichedRelation(entity='Organisati...,An Organisation member is born in a town and t...,10,296,0.181818,0.384615,oneshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
894,894,relations=[EnrichedRelation(entity='ethnic gro...,An ethnic group resides in a populated place. ...,10,297,0.000000,0.341463,oneshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF
895,895,relations=[EnrichedRelation(entity='Organisati...,A band has Organisation members with nationali...,10,298,0.461538,0.384615,oneshot,NousResearch-Hermes-3-Llama-3.2-3B-GGUF


In [29]:
means = df_clean.groupby(["n_nodes", "prompt_type", "model_id"])[
    ["f1_score", "edit_distance"]
].mean()


f1_score  \
n_nodes prompt_type model_id                                            
3       oneshot     NousResearch-Hermes-3-Llama-3.2-3B-GGUF  0.140872   
        zeroshot    NousResearch-Hermes-3-Llama-3.2-3B-GGUF  0.735769   
5       oneshot     NousResearch-Hermes-3-Llama-3.2-3B-GGUF  0.169299   
        zeroshot    NousResearch-Hermes-3-Llama-3.2-3B-GGUF  0.694291   
10      oneshot     NousResearch-Hermes-3-Llama-3.2-3B-GGUF  0.156341   
        zeroshot    NousResearch-Hermes-3-Llama-3.2-3B-GGUF  0.595402   

                                                             edit_distance  
n_nodes prompt_type model_id                                                
3       oneshot     NousResearch-Hermes-3-Llama-3.2-3B-GGUF       0.564505  
        zeroshot    NousResearch-Hermes-3-Llama-3.2-3B-GGUF       0.726824  
5       oneshot     NousResearch-Hermes-3-Llama-3.2-3B-GGUF       0.540193  
        zeroshot    NousResearch-Hermes-3-Llama-3.2-3B-GGUF       0.651715  
10      oneshot     NousResearch-Hermes-3-Llama-3.2-3B-GGUF       0.501106  
        zeroshot    NousResearch-Hermes-3-Llama-3.2-3B-GGUF       0.594902

In [ ]:
means.to_latex()